# LSTM. 5 featerow - mag, x, y, depth, diff_days

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

2023-12-02 16:55:05.651728: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 16:55:05.652290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 16:55:05.653735: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 16:55:05.992680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 16:55:08.900150: W tensorflow/compiler/

In [2]:
ds_train = tf.data.Dataset.load("../data/MEM_ds_train/")
ds_val = tf.data.Dataset.load("../data/MEM_ds_val/")
ds_test = tf.data.Dataset.load("../data/MEM_ds_test/")

2023-12-02 16:55:12.698446: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-02 16:55:13.215981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-02 16:55:13.216072: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-02 16:55:13.225758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-02 16:55:13.225844: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3304064, 465718, 100200)

In [4]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [5]:
def transform_element(features, context, labels):
    return features, (labels[:-1], labels[-1])

In [6]:
ds_train = ds_train.map(transform_element)
ds_val = ds_val.map(transform_element)
ds_test = ds_test.map(transform_element)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(y[0].shape)
    print(y[1].shape)

(1024, 64, 8)
(1024, 4)
(1024,)


In [8]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [9]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(100, n_embed // 8)
        self.embed_dd = tf.keras.layers.Embedding(100, n_embed // 8)
        self.lstm_embed = tf.keras.layers.LSTM(n_embed // 8 * 6, return_sequences=True)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out1 = tf.keras.layers.Dense(4, activation='sigmoid')
        self.out2 = tf.keras.layers.Dense(17, activation='softmax')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, x):
        cont, plate, dd = x[:,:,:-2], x[:,:,-2], x[:,:,-1] 
        x1 = self.lstm_embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x = self.conc([x1, x2, x3])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        out1 = self.out1(x)
        out2 = self.out2(x)
        return [out1, out2]

In [10]:
model = MyModel(128)

In [11]:
model.build(input_shape=(None, BLOCK_SIZE, 8))
model.count_params()

325141

In [12]:
LAMBDA_1 = 2
LAMBDA_2 = 1
loss_weights = {'output_1': LAMBDA_1, 'output_2': LAMBDA_2}

In [13]:
model.compile(optimizer='adam',
            loss={'output_1': 'mse', 'output_2': tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)},
            loss_weights=loss_weights,
            metrics={'output_1': 'mse', 'output_2': 'accuracy'})

In [14]:
history = model.fit(ds_train, epochs=30, validation_data=ds_val)

Epoch 1/30


2023-12-02 16:55:35.824033: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 159 of 2000
2023-12-02 16:55:55.833610: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 490 of 2000
2023-12-02 16:56:15.795896: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 814 of 2000
2023-12-02 16:56:35.831882: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1085 of 2000
2023-12-02 16:56:55.832222: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1397 of 2000
2023-12-02 16:57:05.881377: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take 

3227/3227 [==============================] - 553s 118ms/step - loss: 0.5490 - output_1_loss: 0.0468 - output_2_loss: 0.4554 - output_1_mse: 0.0468 - output_2_accuracy: 0.8533 - val_loss: 0.3879 - val_output_1_loss: 0.0508 - val_output_2_loss: 0.2863 - val_output_1_mse: 0.0508 - val_output_2_accuracy: 0.9061
Epoch 2/30


2023-12-02 17:04:42.763246: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 137 of 2000
2023-12-02 17:05:02.709468: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 424 of 2000
2023-12-02 17:05:12.743173: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 554 of 2000
2023-12-02 17:05:22.755642: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 678 of 2000
2023-12-02 17:05:42.754131: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 952 of 2000
2023-12-02 17:06:02.716235: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a 

1184/3227 [==========>...................] - ETA: 3:55 - loss: 0.5204 - output_1_loss: 0.0427 - output_2_loss: 0.4349 - output_1_mse: 0.0427 - output_2_accuracy: 0.8470

...

3227/3227 [==============================] - 631s 130ms/step - loss: 0.4451 - output_1_loss: 0.0264 - output_2_loss: 0.3923 - output_1_mse: 0.0264 - output_2_accuracy: 0.8565 - val_loss: 0.3524 - val_output_1_loss: 0.0345 - val_output_2_loss: 0.2835 - val_output_1_mse: 0.0345 - val_output_2_accuracy: 0.9062
Epoch 29/30


2023-12-02 22:05:14.633621: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 80 of 2000
2023-12-02 22:05:34.468085: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 267 of 2000
2023-12-02 22:05:44.474573: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 348 of 2000
2023-12-02 22:06:04.467213: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 538 of 2000
2023-12-02 22:06:14.491834: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 618 of 2000
2023-12-02 22:06:34.494827: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 633s 129ms/step - loss: 0.4442 - output_1_loss: 0.0264 - output_2_loss: 0.3914 - output_1_mse: 0.0264 - output_2_accuracy: 0.8566 - val_loss: 0.3535 - val_output_1_loss: 0.0353 - val_output_2_loss: 0.2830 - val_output_1_mse: 0.0353 - val_output_2_accuracy: 0.9063
Epoch 30/30


2023-12-02 22:15:47.736025: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 80 of 2000
2023-12-02 22:16:07.628798: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 250 of 2000
2023-12-02 22:16:17.754747: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 326 of 2000
2023-12-02 22:16:37.661958: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 516 of 2000
2023-12-02 22:16:57.696183: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 694 of 2000
2023-12-02 22:17:17.675039: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3227/3227 [==============================] - 650s 131ms/step - loss: 0.4425 - output_1_loss: 0.0263 - output_2_loss: 0.3899 - output_1_mse: 0.0263 - output_2_accuracy: 0.8568 - val_loss: 0.3527 - val_output_1_loss: 0.0344 - val_output_2_loss: 0.2840 - val_output_1_mse: 0.0344 - val_output_2_accuracy: 0.9057


In [15]:
model.evaluate(ds_test)

98/98 [==============================] - 12s 118ms/step - loss: 0.3576 - output_1_loss: 0.0365 - output_2_loss: 0.2846 - output_1_mse: 0.0365 - output_2_accuracy: 0.9083


[0.3576006293296814,
 0.03650077432394028,
 0.2845989763736725,
 0.03650077432394028,
 0.9082934260368347]

In [16]:
model.save("../models/MEM_model_v1.keras")

In [17]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/MEM_history_model_v1.csv", index=False)